<h4 style="font-variant-caps: small-caps;font-size:35pt;">Databricks-ML-professional-S01b-Experiment-Tracking</h4>

<div style='background-color:black;border-radius:5px;border-top:1px solid'></div>
<br/>
<p>This Notebook adds information related to the following requirements:</p><br/>
<b>Experiment Tracking:</b>
<ul>
<li>Manually log parameters, models, and evaluation metrics using MLflow</li>
<li>Programmatically access and use data, metadata, and models from MLflow experiments</li>
</ul>
<br/>
<p><b>Download this notebook at format ipynb <a href="Databricks-ML-professional-S01b-Experiment-Tracking.ipynb">here</a>.</b></p>
<br/>
<div style='background-color:black;border-radius:5px;border-top:1px solid'></div>

<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">1. Import libraries</span></div>

In [ ]:
import pandas as pd
import seaborn as sns
#
from pyspark.sql.functions import *
#
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
#
import mlflow
#
import logging

In [ ]:
logging.getLogger("mlflow").setLevel(logging.FATAL)

<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">2. Load dataset, convert to Spark DataFrame</span></div>

In [ ]:
tips_df = sns.load_dataset("tips")
#
tips_sdf = spark.createDataFrame(tips_df)
#
display(tips_sdf.limit(5))

total_bill,tip,sex,smoker,day,time,size
16.99,1.01,Female,No,Sun,Dinner,2
10.34,1.66,Male,No,Sun,Dinner,3
21.01,3.5,Male,No,Sun,Dinner,3
23.68,3.31,Male,No,Sun,Dinner,2
24.59,3.61,Female,No,Sun,Dinner,4


In [ ]:
display(tips_sdf.filter("size is null"))

total_bill,tip,sex,smoker,day,time,size


<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">3. Prepare data</span></div>

<p>Some transformations are done to prepare dataset to be used in training a ML model.</p>
<table border style='border-collapse: collapse;'>
<tr style="background-color:#EDEDED">
    <th>column name</th>
    <th>comment</th>
</tr>
<tr>
    <td><code>tip</code></td>
    <td><b style='color:orangered'>target</b> to predict. Contains numeric</td>
</tr>
<tr>
    <td><code>total_bill</code></td>
    <td>numeric column to keep as is</td>
</tr>
<tr>
    <td><code>sex</code></td>
    <td>Contains <code>Female</code> and <code>Male</code> converted to <code>0</code> and <code>1</code></td>
</tr>
<tr>
    <td><code>smoker</code></td>
    <td>Contains <code>yes</code> and <code>no</code> converted to <code>0</code> and <code>1</code></td>
</tr>
<tr>
    <td><code>time</code></td>
    <td>Contains <code>Dinner</code> and <code>Lunch</code> converted to <code>0</code> and <code>1</code></td>
</tr>
<tr>
    <td><code>day</code></td>
    <td>categorical column to <b>One Hot Encode</b></td>
</tr>
<tr>
    <td><code>size</code></td>
    <td>categorical column to <b>One Hot Encode</b></td>
</tr>
</table>

In [ ]:
tips_sdf = tips_sdf.selectExpr("total_bill",
                               "tip",
                               "case when sex = 'Female' then 1 else 0 end as sex",
                               "case when smoker = 'yes' then 1 else 0 end as smoker",
                               "case when time = 'Dinner' then 1 else 0 end as time",
                               "day",
                               "size")
#
train_df, test_df = tips_sdf.randomSplit([.8, .2])
#
ohe_cols = ["size", "day"]
num_cols = ["total_bill", "sex", "smoker", "time"]
target_col = "tip"
#
string_indexer = StringIndexer(inputCols=ohe_cols, outputCols=[c+"_index" for c in ohe_cols], handleInvalid="skip")
#
ohe = OneHotEncoder()
ohe.setInputCols([c+"_index" for c in ohe_cols])
ohe.setOutputCols([c+"_ohe" for c in ohe_cols])
#
assembler_inputs = [c+"_ohe" for c in ohe_cols] + num_cols
vec_assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">4. Evaluator and model</span></div>

In [ ]:
gbt =       GBTRegressor(featuresCol="features", labelCol=target_col, maxIter=5)
evaluator = RegressionEvaluator(labelCol=target_col, predictionCol="prediction", metricName="rmse")

<a id="manuallylog"></a>
<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">5. Manually log parameters, models, and evaluation metrics using MLflow</span></div>

In [ ]:
model_name = "GBT-Regressor"
#
with mlflow.start_run(run_name="Tip-run") as run:
    #
    # define pipeline stages according to model
    stages = [string_indexer, ohe, vec_assembler, gbt]
    #
    # set pipeline
    pipeline = Pipeline(stages=stages)
    #
    # fit pipeline to train set
    model = pipeline.fit(train_df)
    #
    # manually log model to mlflow
    mlflow.spark.log_model(model, model_name)
    #
    # manually log parameter to mlflow
    mlflow.log_param("maxIter", 5)
    #
    # predict test set
    pred_df = model.transform(test_df)
    #
    # evaluate prediction
    rmse = evaluator.evaluate(pred_df)
    #
    # manually log metric to mlflow
    mlflow.log_metric("rmse", rmse)

<a id="programmaticallyaccess"></a>
<div style='background-color:rgba(30, 144, 255, 0.1);border-radius:5px;padding:2px;'>
<span style="font-variant-caps: small-caps;font-weight:700">6. Programmatically access and use data, metadata, and models from MLflow experiments</span></div>

<p>This can be done in different ways. One of them is to access it programmaticaly with the function <code>mlflow.search_runs</code> which results in a Pandas dataframe containing all useful information for all runs in the current experiment <i>(by default, the current experiment has the name of the current notebook)</i>:</p>

In [ ]:
mlflow.search_runs().drop(['tags.mlflow.databricks.workspaceURL',
                           'tags.mlflow.databricks.notebookPath',
                           'tags.mlflow.source.name',
                           'tags.mlflow.user'], axis=1)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.rmse,params.maxIter,tags.mlflow.databricks.cluster.libraries.error,tags.mlflow.databricks.notebookRevisionID,tags.mlflow.databricks.workspaceID,tags.mlflow.databricks.notebookID,tags.sparkDatasourceInfo,tags.mlflow.log-model.history,tags.mlflow.databricks.notebook.commandID,tags.mlflow.source.type,tags.mlflow.databricks.webappURL,tags.mlflow.runName,tags.mlflow.databricks.cluster.info,tags.mlflow.databricks.cluster.id
0,3b461072c96e4ce492250c86e5d4b04a,4106912166953874,FINISHED,dbfs:/databricks/mlflow-tracking/4106912166953...,2023-11-22 16:47:43.155000+00:00,2023-11-22 16:48:13.660000+00:00,1.685712,5,This message class grpc_shaded.com.databricks....,1700671694065,3607579860940718,4106912166953874,path=mlflowdbfs:/artifacts?run_id=05c39cdd803d...,"[{""artifact_path"":""GBT-Regressor"",""flavors"":{""...",4723686315041778941_5042614833210616149_6f6f9d...,NOTEBOOK,https://eastus-c3.azuredatabricks.net,Tip-run,"{""cluster_name"":""Victor Bonnet's Cluster"",""spa...",1027-081006-5cgi5kuh
1,05c39cdd803d41aaaf1d95e938bccb3c,4106912166953874,FINISHED,dbfs:/databricks/mlflow-tracking/4106912166953...,2023-11-22 16:43:02.360000+00:00,2023-11-22 16:43:32.223000+00:00,1.071217,5,This message class grpc_shaded.com.databricks....,1700671412555,3607579860940718,4106912166953874,None,"[{""artifact_path"":""GBT-Regressor"",""flavors"":{""...",4723686315041778941_8003943757913343583_2f0694...,NOTEBOOK,https://eastus-c3.azuredatabricks.net,Tip-run,"{""cluster_name"":""Victor Bonnet's Cluster"",""spa...",1027-081006-5cgi5kuh
2,197b10148baf4c55a9cfd55cb716a45f,4106912166953874,FINISHED,dbfs:/databricks/mlflow-tracking/4106912166953...,2023-11-22 16:42:20.492000+00:00,2023-11-22 16:42:50.008000+00:00,1.568573,5,This message class grpc_shaded.com.databricks....,1700671370330,3607579860940718,4106912166953874,None,"[{""artifact_path"":""GBT-Regressor"",""flavors"":{""...",4723686315041778941_8672488673836230069_a44c4b...,NOTEBOOK,https://eastus-c3.azuredatabricks.net,Tip-run,"{""cluster_name"":""Victor Bonnet's Cluster"",""spa...",1027-081006-5cgi5kuh
3,0f53885013194fdab9bc8ad6383e7bdf,4106912166953874,FINISHED,dbfs:/databricks/mlflow-tracking/4106912166953...,2023-11-22 16:37:23.545000+00:00,2023-11-22 16:37:53.810000+00:00,1.581776,5,This message class grpc_shaded.com.databricks....,1700671074137,3607579860940718,4106912166953874,None,"[{""artifact_path"":""GBT-Regressor"",""flavors"":{""...",4723686315041778941_6064693358274921191_17ebf6...,NOTEBOOK,https://eastus-c3.azuredatabricks.net,Tip-run,"{""cluster_name"":""Victor Bonnet's Cluster"",""spa...",1027-081006-5cgi5kuh
4,15ba91e309d548c2a85c1f01c006e39b,4106912166953874,FINISHED,dbfs:/databricks/mlflow-tracking/4106912166953...,2023-11-22 16:35:48.939000+00:00,2023-11-22 16:36:25.395000+00:00,1.498592,5,This message class grpc_shaded.com.databricks....,1700670985719,3607579860940718,4106912166953874,None,"[{""artifact_path"":""GBT-Regressor"",""flavors"":{""...",4723686315041778941_8549608798050021702_4078e9...,NOTEBOOK,https://eastus-c3.azuredatabricks.net,Tip-run,"{""cluster_name"":""Victor Bonnet's Cluster"",""spa...",1027-081006-5cgi5kuh


<p>Using Pandas syntax information can be filtered on what is needed:</p>

In [ ]:
mlflow.search_runs()[["tags.mlflow.runName", "run_id", "params.maxIter", "metrics.rmse"]].sort_values(by=['metrics.rmse'], ascending=True)

,tags.mlflow.runName,run_id,params.maxIter,metrics.rmse
1,Tip-run,05c39cdd803d41aaaf1d95e938bccb3c,5,1.071217
4,Tip-run,15ba91e309d548c2a85c1f01c006e39b,5,1.498592
2,Tip-run,197b10148baf4c55a9cfd55cb716a45f,5,1.568573
3,Tip-run,0f53885013194fdab9bc8ad6383e7bdf,5,1.581776
0,Tip-run,3b461072c96e4ce492250c86e5d4b04a,5,1.685712


<p>A <b>SQL filter</b> can also be applied directly in the <code>mlflow.search_run()</code> function by using its <code>filter_string</code> parameter. This is particularly useful when there are many runs:</p>

In [ ]:
mlflow.search_runs(filter_string="tags.mlflow.runName like '%Tip%' and metrics.rmse<=1.5")[["tags.mlflow.runName", "run_id", "params.maxIter", "metrics.rmse"]]

,tags.mlflow.runName,run_id,params.maxIter,metrics.rmse
0,Tip-run,05c39cdd803d41aaaf1d95e938bccb3c,5,1.071217
1,Tip-run,15ba91e309d548c2a85c1f01c006e39b,5,1.498592


<p>With this, let's load the best model:</p>

In [ ]:
bestModelRunId = mlflow.search_runs().sort_values(by=['metrics.rmse'], ascending=True).head(1)["run_id"].values[0]
#
best_model_path = f"runs:/{bestModelRunId}/{model_name}"
print(f"Best model path is: {best_model_path}")
#
loaded_model = mlflow.spark.load_model(best_model_path)

Best model path is: runs:/05c39cdd803d41aaaf1d95e938bccb3c/GBT-Regressor


In [ ]:
display(loaded_model.transform(test_df).select("tip", "prediction"))

tip,prediction
1.32,2.2600362250689874
1.56,2.2600362250689874
1.57,2.270591780624543
2.0,2.4706720284816104
1.01,2.621464796497559
3.0,3.2463005089207058
2.5,3.1992041196791403
3.31,2.9060591599135535
3.12,4.0487121332066645
7.58,4.6646880493827165


<img src="https://i.ibb.co/xSdfvyD/mlflow3.png"/>